In [1]:
import pandas as pd

In [2]:
#Load data
train_data = pd.read_csv('D:\\study\\train.csv')
test_data = pd.read_csv('D:\\study\\test.csv')

In [3]:
Sex = []
for i in range(len(train_data)):
    if(train_data.ix[i,'Sex'] == 'female'):
        Sex.append(0)
    else:
        Sex.append(1)
train_data.drop('Sex', axis=1, inplace=True)
train_data['Sex'] = Sex
train_data.drop(['Name','Cabin','Ticket'], axis=1, inplace=True)
a = list(train_data['Embarked'])
Embarked=[]
for i in a:
    if i == 'S':
        Embarked.append(0)
    elif i == 'Q':
        Embarked.append(1)
    elif i == 'C':
        Embarked.append(2)
    else:
        Embarked.append(None)

D:\ruanjian\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
train_data.drop(['Embarked'], axis=1, inplace=True)
train_data['Embarked'] = Embarked
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(train_data)
data = imr.transform(train_data.values)
train = pd.DataFrame(data)
train.columns = train_data.columns

In [6]:
from hpsklearn import HyperoptEstimator, any_classifier
from hyperopt import tpe
import numpy as np
X = np.array(train.ix[:,['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']])
y = np.array(train.ix[:, 'Survived'])
test_size = int( 0.2 * len( y ) )
seed = 1234
np.random.seed( seed )
indices = np.random.permutation(len(X))
X_train = X[ indices[:-test_size]]
y_train = y[ indices[:-test_size]]
X_test = X[ indices[-test_size:]]
y_test = y[ indices[-test_size:]]

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


D:\ruanjian\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
D:\ruanjian\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [7]:
estim = HyperoptEstimator(classifier=any_classifier('clf'), algo=tpe.suggest, trial_timeout=300)
estim.fit(X_train, y_train)

100%|████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.32s/it, best loss: 0.18181818181818177]


In [8]:
print(estim.score(X_test, y_test))

0.8089887640449438


In [9]:
print(estim.best_model())

{'learner': RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.26047484622312844,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=11, n_jobs=1, oob_score=False, random_state=1,
            verbose=False, warm_start=False), 'preprocs': (MinMaxScaler(copy=True, feature_range=(-1.0, 1.0)),), 'ex_preprocs': ()}


In [10]:
#test data
Sex = []
for i in range(len(test_data)):
    if(test_data.ix[i,'Sex'] == 'female'):
        Sex.append(0)
    else:
        Sex.append(1)
test_data.drop('Sex', axis=1, inplace=True)
test_data['Sex'] = Sex
test_data.drop(['Name','Cabin','Ticket'], axis=1, inplace=True)
a = list(test_data['Embarked'])
Embarked=[]
for i in a:
    if i == 'S':
        Embarked.append(0)
    elif i == 'Q':
        Embarked.append(1)
    elif i == 'C':
        Embarked.append(2)
    else:
        Embarked.append(None)

D:\ruanjian\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [11]:
test_data.drop(['Embarked'], axis=1, inplace=True)
test_data['Embarked'] = Embarked
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(test_data)
data = imr.transform(test_data.values)
test = pd.DataFrame(data)
test.columns = test_data.columns

In [12]:
X_test = np.array(test.ix[:,['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']])

D:\ruanjian\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [15]:
predict_data = estim.predict(X_test)

In [17]:
import csv
def save2csv(PassengerId, survival):
    with open(r'E:/Titanic.csv','a+',newline='',encoding='utf_8_sig') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(len(predict_data)):
            writer.writerow([PassengerId[i], survival[i]])

In [18]:
save2csv(test_data['PassengerId'], predict_data)